## R과 MySQL을 활용한
# 영화 데이터를 이용한 DW구축 실습
* 충북대학교 빅데이터센터
* 작성자 : 이경희
* 작성일자 : '19.03.13

In [1]:
# R 버전 확인하기
version

               _                           
platform       x86_64-w64-mingw32          
arch           x86_64                      
os             mingw32                     
system         x86_64, mingw32             
status                                     
major          3                           
minor          5.2                         
year           2018                        
month          12                          
day            20                          
svn rev        75870                       
language       R                           
version.string R version 3.5.2 (2018-12-20)
nickname       Eggshell Igloo              

In [2]:
# 필요한 라이브러리 불러오기
# install.packages(c("DBI", "RMySQL", "dplyr", "readxl"))
library(DBI) # SQL 쿼리 관련 라이브러리
library(RMySQL) # MySQL과 R을 연결하는 라이브러리
library(dplyr) # 데이터 처리를 위한 라이브러리
library(readxl) # Excel 데이터를 불러오기 위한 라이브러리


Attaching package: 'dplyr'

The following objects are masked from 'package:stats':

    filter, lag

The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union



In [3]:
# 라이브러리들이 로딩되었는지 확인
search()

[1] ".GlobalEnv"        "package:readxl"    "package:dplyr"    
 [4] "package:RMySQL"    "package:DBI"       "jupyter:irkernel" 
 [7] "package:stats"     "package:graphics"  "package:grDevices"
[10] "package:utils"     "package:datasets"  "package:methods"  
[13] "Autoloads"         "package:base"

In [4]:
# 계정 연결하기
con <- dbConnect(MySQL(), user="movie", password="moviegoer", 
                 host="localhost", port=13306); con

<MySQLConnection:0,0>

In [5]:
# Ddatabase Schema 생성
dbGetQuery(con, "create database if not exists movie default character set euckr")

<0 x 0 matrix>

In [6]:
# DB Schema 선택
dbGetQuery(con, "use movie")

<0 x 0 matrix>

In [7]:
# excel 파일 불러오기
movie_total <- read_excel("./data/Movie_data/1. 활용 DataSet_ 영화정보(2012.1~2013.10).xlsx")
movie_day_2012 <- read_excel("./data/Movie_data/2-1. 활용 DataSet_ 일별 박스 오피스(2012).xlsx")
movie_day_2013 <- read_excel("./data/Movie_data/2-2. 활용 DataSet_ 일별 박스 오피스(2013).xlsx")
movie_weather <- read_excel("./data/Movie_data/3. 활용 DataSet_ 일자별 날씨.xlsx")
movie_news <- read_excel("./data/Movie_data/4. 활용 DataSet_ 뉴스 건수.xlsx")
movie_reply <- read_excel("./data/Movie_data/5. 활용 DataSet_ 주별 댓글수.xlsx")

Warning message in read_fun(path = enc2native(normalizePath(path)), sheet_i = sheet, :
"Coercing text to numeric in B1016 / R1016C2: '107070'"Warning message in read_fun(path = enc2native(normalizePath(path)), sheet_i = sheet, :
"Coercing text to numeric in B1020 / R1020C2: '84936'"Warning message in read_fun(path = enc2native(normalizePath(path)), sheet_i = sheet, :
"Coercing text to numeric in C10277 / R10277C3: '107070'"Warning message in read_fun(path = enc2native(normalizePath(path)), sheet_i = sheet, :
"Coercing text to numeric in C10345 / R10345C3: '107070'"Warning message in read_fun(path = enc2native(normalizePath(path)), sheet_i = sheet, :
"Coercing text to numeric in C10417 / R10417C3: '107070'"Warning message in read_fun(path = enc2native(normalizePath(path)), sheet_i = sheet, :
"Coercing text to numeric in C10494 / R10494C3: '107070'"Warning message in read_fun(path = enc2native(normalizePath(path)), sheet_i = sheet, :
"Coercing text to numeric in C10562 / R10562C3: '10707

In [8]:
# 영화 정보 파일 확인
str(movie_total)

Classes 'tbl_df', 'tbl' and 'data.frame':	1022 obs. of  20 variables:
 $ 순번        : num  1 2 3 4 5 6 7 8 9 10 ...
 $ Code        : num  78726 83893 72363 88253 72054 ...
 $ 영화명      : chr  "도둑들" "광해, 왕이된남자" "어벤져스" "늑대소년" ...
 $ 감독        : chr  "최동훈" "추창민" "조스웨던" "조성희" ...
 $ 제작사      : chr  "(주)케이퍼필름" "리얼라이즈픽쳐스(주),씨제이이앤엠주식회사" NA "(주)영화사비단길" ...
 $ 수입사      : chr  NA NA "한국소니픽쳐스릴리징브에나비스타영화㈜" NA ...
 $ 배급사      : chr  "쇼박스㈜미디어플렉스" "씨제이이앤엠주식회사" "한국소니픽쳐스릴리징브에나비스타영화㈜" "씨제이이앤엠주식회사" ...
 $ 개봉일      : chr  "2012-07-25" "2012-09-13" "2012-04-26" "2012-10-31" ...
 $ 영화유형    : chr  "개봉영화" "개봉영화" "개봉영화" "개봉영화" ...
 $ 영화형태    : chr  "장편" "장편" "장편" "장편" ...
 $ 국적        : chr  "한국" "한국" "미국" "한국" ...
 $ 전국스크린수: num  1072 810 967 706 1210 ...
 $ 전국매출액  : chr  "93664808500" "88899448769" "59557853478" "46590107000" ...
 $ 전국관객수  : num  12983178 12319390 7074867 6654390 6396528 ...
 $ 서울매출액  : chr  "26089426500" "25192804727" "19280107897" "12336313500" ...
 $ 서울관객수  : num  3468906 3355700 2147619 16

In [9]:
# 변수명 바꾸기
names(movie_total) <- c("idx", "code", "m_name", "director", "productor", "incomer", "distributor", "open", "type1", "type2", 
                        "nation", "the_number_of_screens", "nation_income", "nation_audiences",
                       "seoul_income", "seoul_audiences", "style", "level", "story", "running_time")
str(movie_total)

Classes 'tbl_df', 'tbl' and 'data.frame':	1022 obs. of  20 variables:
 $ idx                  : num  1 2 3 4 5 6 7 8 9 10 ...
 $ code                 : num  78726 83893 72363 88253 72054 ...
 $ m_name               : chr  "도둑들" "광해, 왕이된남자" "어벤져스" "늑대소년" ...
 $ director             : chr  "최동훈" "추창민" "조스웨던" "조성희" ...
 $ productor            : chr  "(주)케이퍼필름" "리얼라이즈픽쳐스(주),씨제이이앤엠주식회사" NA "(주)영화사비단길" ...
 $ incomer              : chr  NA NA "한국소니픽쳐스릴리징브에나비스타영화㈜" NA ...
 $ distributor          : chr  "쇼박스㈜미디어플렉스" "씨제이이앤엠주식회사" "한국소니픽쳐스릴리징브에나비스타영화㈜" "씨제이이앤엠주식회사" ...
 $ open                 : chr  "2012-07-25" "2012-09-13" "2012-04-26" "2012-10-31" ...
 $ type1                : chr  "개봉영화" "개봉영화" "개봉영화" "개봉영화" ...
 $ type2                : chr  "장편" "장편" "장편" "장편" ...
 $ nation               : chr  "한국" "한국" "미국" "한국" ...
 $ the_number_of_screens: num  1072 810 967 706 1210 ...
 $ nation_income        : chr  "93664808500" "88899448769" "59557853478" "46590107000" ...
 $ nation_audiences     : 

In [10]:
# 2012년 일별 박스오피스 확인
str(movie_day_2012)

Classes 'tbl_df', 'tbl' and 'data.frame':	18815 obs. of  25 variables:
 $ rank       : num  2 7 14 16 21 31 50 2 7 13 ...
 $ datecode   : chr  "84899-20120107" "82253-20120107" "90388-20120107" "78569-20120107" ...
 $ code       : num  84899 82253 90388 78569 53746 ...
 $ m_name     : chr  "원더풀 라디오" "다크아워" "코알라 키드 : 영웅의 탄생" "치코와 리타" ...
 $ open       : POSIXct, format: "2012-01-05" "2012-01-05" ...
 $ s_price    : num  1.09e+09 3.25e+08 2.02e+07 1.09e+07 2.08e+06 ...
 $ date       : POSIXct, format: "2012-01-07" "2012-01-07" ...
 $ s_percent  : num  0.173 0.052 0.003 0.002 0 0 0 0.18 0.062 0.006 ...
 $ s_increment: num  5.90e+08 1.55e+08 1.54e+07 6.16e+06 1.13e+06 ...
 $ i_percent  : num  1.2 0.9 3.2 1.3 1.2 4 0.1 0.2 -0.1 1 ...
 $ s_sum      : num  2.08e+09 6.75e+08 3.44e+07 2.14e+07 3.84e+06 ...
 $ people     : num  144597 39040 2452 1303 241 ...
 $ p_increment: num  78970 18742 1611 735 124 ...
 $ p_percent  : num  1.2 0.9 1.9 1.3 1.1 3.9 0.1 0.1 -0.1 0.8 ...
 $ p_sum      : num  28

In [11]:
# 2013년 일별 박스오피스 확인
str(movie_day_2013)

Classes 'tbl_df', 'tbl' and 'data.frame':	12687 obs. of  25 variables:
 $ rank       : num  1 2 4 5 6 7 8 9 10 11 ...
 $ datecode   : chr  "83250-20130107" "89755-20130107" "91045-20130107" "87307-20130107" ...
 $ code       : num  83250 89755 91045 87307 68073 ...
 $ m_name     : chr  "타워" "레미제라블" "반창꼬" "주먹왕 랄프" ...
 $ open       : POSIXct, format: "2012-12-25" "2012-12-19" ...
 $ s_price    : num  8.08e+08 5.21e+08 2.55e+08 7.12e+07 7.94e+07 ...
 $ date       : POSIXct, format: "2013-01-07" "2013-01-07" ...
 $ s_percent  : num  0.331 0.213 0.105 0.029 0.033 ...
 $ s_increment: num  -1.55e+09 -9.63e+08 -4.39e+08 -1.98e+08 -1.52e+08 ...
 $ i_percent  : num  -0.7 -0.6 -0.6 -0.7 -0.7 -0.7 -0.7 -0.7 -0.5 -0.7 ...
 $ s_sum      : num  2.67e+10 3.14e+10 1.64e+10 5.87e+09 2.32e+10 ...
 $ people     : num  114309 74536 36225 10474 10197 ...
 $ p_increment: num  -204802 -125045 -56501 -26440 -18193 ...
 $ p_percent  : num  -0.6 -0.6 -0.6 -0.7 -0.6 -0.7 -0.6 -0.7 -0.4 -0.7 ...
 $ p_sum      : n

In [12]:
# 일자별 날씨 확인
str(movie_weather)

Classes 'tbl_df', 'tbl' and 'data.frame':	684 obs. of  3 variables:
 $ DateKey: POSIXct, format: "2012-01-01" "2012-01-02" ...
 $ 강수량 : num  0 0 0.4 0 0 0 0 0 0 0 ...
 $ 요일   : chr  "일요일" "월요일" "화요일" "수요일" ...


In [13]:
# 변수명 바꾸기 및 변수 속성 바꾸기
names(movie_weather) <- c("datekey", "rainfall", "day")
movie_weather$datekey <- as.character(movie_weather$datekey)
str(movie_weather)

Classes 'tbl_df', 'tbl' and 'data.frame':	684 obs. of  3 variables:
 $ datekey : chr  "2012-01-01" "2012-01-02" "2012-01-03" "2012-01-04" ...
 $ rainfall: num  0 0 0.4 0 0 0 0 0 0 0 ...
 $ day     : chr  "일요일" "월요일" "화요일" "수요일" ...


In [14]:
# 뉴스 확인
str(movie_news)

Classes 'tbl_df', 'tbl' and 'data.frame':	865 obs. of  3 variables:
 $ CODE      : chr  "116104" "114791" "111118" "111008" ...
 $ 영화명    : chr  "멋진 녀석들" "사티아그라하" "스트리퍼 VS 늑대인간" "스티븐 시걸의 리벤지" ...
 $ 뉴스(건수): num  5 4 0 1 26 0 0 0 0 0 ...


In [15]:
# 변수명 바꾸기
names(movie_news) <- c("code", "m_name", "news_count")
str(movie_news)

Classes 'tbl_df', 'tbl' and 'data.frame':	865 obs. of  3 variables:
 $ code      : chr  "116104" "114791" "111118" "111008" ...
 $ m_name    : chr  "멋진 녀석들" "사티아그라하" "스트리퍼 VS 늑대인간" "스티븐 시걸의 리벤지" ...
 $ news_count: num  5 4 0 1 26 0 0 0 0 0 ...


In [16]:
# 주별 댓글수 확인
str(movie_reply)

Classes 'tbl_df', 'tbl' and 'data.frame':	751 obs. of  5 variables:
 $ m_name(영화명): chr  "007 스카이폴" "10 아이템오어레스" "1999, 면회" "19곰테드" ...
 $ 1주차         : num  75 1 0 20 102 1 3 3 1 38 ...
 $ 2주차         : num  47 0 0 31 135 6 3 4 1 79 ...
 $ 3주차         : num  23 0 2 19 1208 ...
 $ 4주차         : num  28 0 0 10 13216 ...


In [17]:
# 변수명 바꾸기
names(movie_reply) <- c("m_name", "first_week", "second_week", "third_week", "fourth_week")
str(movie_reply)

Classes 'tbl_df', 'tbl' and 'data.frame':	751 obs. of  5 variables:
 $ m_name     : chr  "007 스카이폴" "10 아이템오어레스" "1999, 면회" "19곰테드" ...
 $ first_week : num  75 1 0 20 102 1 3 3 1 38 ...
 $ second_week: num  47 0 0 31 135 6 3 4 1 79 ...
 $ third_week : num  23 0 2 19 1208 ...
 $ fourth_week: num  28 0 0 10 13216 ...


In [18]:
# movie_day 합치기
movie_day <- rbind(movie_day_2012, movie_day_2013)
print(dim(movie_day_2012))
print(dim(movie_day_2013))
print(dim(movie_day))

[1] 18815    25
[1] 12687    25
[1] 31502    25


In [19]:
# movie_day에서 code null값 확인
print(sum(is.null(movie_day$code)))
print(sum(is.na(movie_day$code)))

[1] 0
[1] 0


In [20]:
# movie_day 변수의 datekey와 code의 형식을 character로 변경
movie_day$code <- as.character(movie_day$code)
movie_day$datecode <- as.character(movie_day$datecode)
str(movie_day)

Classes 'tbl_df', 'tbl' and 'data.frame':	31502 obs. of  25 variables:
 $ rank       : num  2 7 14 16 21 31 50 2 7 13 ...
 $ datecode   : chr  "84899-20120107" "82253-20120107" "90388-20120107" "78569-20120107" ...
 $ code       : chr  "84899" "82253" "90388" "78569" ...
 $ m_name     : chr  "원더풀 라디오" "다크아워" "코알라 키드 : 영웅의 탄생" "치코와 리타" ...
 $ open       : POSIXct, format: "2012-01-05" "2012-01-05" ...
 $ s_price    : num  1.09e+09 3.25e+08 2.02e+07 1.09e+07 2.08e+06 ...
 $ date       : POSIXct, format: "2012-01-07" "2012-01-07" ...
 $ s_percent  : num  0.173 0.052 0.003 0.002 0 0 0 0.18 0.062 0.006 ...
 $ s_increment: num  5.90e+08 1.55e+08 1.54e+07 6.16e+06 1.13e+06 ...
 $ i_percent  : num  1.2 0.9 3.2 1.3 1.2 4 0.1 0.2 -0.1 1 ...
 $ s_sum      : num  2.08e+09 6.75e+08 3.44e+07 2.14e+07 3.84e+06 ...
 $ people     : num  144597 39040 2452 1303 241 ...
 $ p_increment: num  78970 18742 1611 735 124 ...
 $ p_percent  : num  1.2 0.9 1.9 1.3 1.1 3.9 0.1 0.1 -0.1 0.8 ...
 $ p_sum      : num  

In [21]:
# movie_total에서 code null값 확인
print(sum(is.null(movie_total$code)))
print(sum(is.na(movie_total$code)))

[1] 0
[1] 157


In [22]:
# movie_total에서 code 값이 na인 값을 제거
movie_total <- movie_total[!is.na(movie_total$code),]
str(movie_total)

Classes 'tbl_df', 'tbl' and 'data.frame':	865 obs. of  20 variables:
 $ idx                  : num  1 2 3 4 5 6 7 8 9 10 ...
 $ code                 : num  78726 83893 72363 88253 72054 ...
 $ m_name               : chr  "도둑들" "광해, 왕이된남자" "어벤져스" "늑대소년" ...
 $ director             : chr  "최동훈" "추창민" "조스웨던" "조성희" ...
 $ productor            : chr  "(주)케이퍼필름" "리얼라이즈픽쳐스(주),씨제이이앤엠주식회사" NA "(주)영화사비단길" ...
 $ incomer              : chr  NA NA "한국소니픽쳐스릴리징브에나비스타영화㈜" NA ...
 $ distributor          : chr  "쇼박스㈜미디어플렉스" "씨제이이앤엠주식회사" "한국소니픽쳐스릴리징브에나비스타영화㈜" "씨제이이앤엠주식회사" ...
 $ open                 : chr  "2012-07-25" "2012-09-13" "2012-04-26" "2012-10-31" ...
 $ type1                : chr  "개봉영화" "개봉영화" "개봉영화" "개봉영화" ...
 $ type2                : chr  "장편" "장편" "장편" "장편" ...
 $ nation               : chr  "한국" "한국" "미국" "한국" ...
 $ the_number_of_screens: num  1072 810 967 706 1210 ...
 $ nation_income        : chr  "93664808500" "88899448769" "59557853478" "46590107000" ...
 $ nation_audiences     : n

In [23]:
a <- dbWriteTable(con, name = "movie_total", value = movie_total, append = T, row.names=F)
a

[1] TRUE

In [24]:
a <- dbWriteTable(con, name = "movie_day", value = movie_day, append = T, row.names=F)
a

[1] TRUE

In [25]:
a <- dbWriteTable(con, name = "movie_weather", value = movie_weather, append = T, row.names=F)
a

[1] TRUE

In [26]:
a <- dbWriteTable(con, name = "movie_news", value = movie_news, append = T, row.names=F)
a

[1] TRUE

In [27]:
a <- dbWriteTable(con, name = "movie_reply", value = movie_reply, append = T, row.names=F)
a

[1] TRUE

In [28]:
# FactTable 만들기
fact_table <- "CREATE TABLE FactTable (
                code VARCHAR(255),
                m_name VARCHAR(255),
                open DATETIME,
                s_price FLOAT,
                datekey VARCHAR(255),
                s_percent FLOAT,
                s_increment FLOAT,
                i_percent FLOAT,
                s_sum FLOAT,
                people FLOAT,
                p_increment FLOAT,
                p_percent FLOAT,
                p_sum FLOAT,
                screen FLOAT,
                count FLOAT
            )"

dbGetQuery(con, fact_table)

<0 x 0 matrix>

In [30]:
# FactTable에 movie_day 데이터 insert
insert_movie_day_to_fact <- "INSERT INTO FactTable
                            SELECT code, m_name, open, s_price, date as datekey, s_percent, s_increment,
                                    i_percent, s_sum, people, p_increment, p_percent, p_sum, screen, count
                            FROM movie_day"
dbGetQuery(con, insert_movie_day_to_fact)

<0 x 0 matrix>

In [31]:
# DimMovie 만들기
dim_movie <- "CREATE TABLE DimMovie(
                code VARCHAR(255),
                m_name VARCHAR(255),
                director VARCHAR(255),
                productor VARCHAR(255),
                incomer VARCHAR(255),
                distributor VARCHAR(255),
                open VARCHAR(255),
                type1 VARCHAR(255),
                type2 VARCHAR(255),
                nation VARCHAR(255),
                style VARCHAR(255),
                level VARCHAR(255),
                story TEXT
            )"
dbGetQuery(con, dim_movie)

<0 x 0 matrix>

In [32]:
# DimMovie 테이블에 movie_total 데이터 insert
insert_movie_total_to_dimmovie <- "INSERT INTO DimMovie(
                                    SELECT code, m_name, director, productor, incomer, distributor, open, 
                                            type1, type2, nation, style, level, story
                                    FROM movie_total
                                )"
dbGetQuery(con, insert_movie_total_to_dimmovie)

<0 x 0 matrix>

In [33]:
# 컬럼 붙이기 
movie_total$action <- ""
movie_total$history <- ""
movie_total$melo <- ""
movie_total$comedy <- ""
movie_total$sexual <- ""
movie_total$crime <- ""
movie_total$horror <- ""
movie_total$drama <- ""
movie_total$adventure <- ""
movie_total$mystery <- ""
movie_total$thrill <- ""
movie_total$sf <- ""
movie_total$animation <- ""
movie_total$fantasy <- ""
movie_total$family <- ""
movie_total$war <- ""
movie_total$musical <- ""
movie_total$daqu <- ""
movie_total$play <- ""
movie_total$etc <- ""
movie_total$western <- ""
print(dim(movie_total))

[1] 865  41


In [34]:
# movie_total 테이블에 장르 구분
for(i in 1:NROW(movie_total)) {
  if(grepl("액션", movie_total$style[i])) {
    movie_total$action[i] <- "1"
  } else {
    movie_total$action[i] <- "0"
  }
  if(grepl("사극", movie_total$style[i])) {
    movie_total$history[i] <- "1"
  } else {
    movie_total$history[i] <- "0"
  }
  if(grepl("멜로", movie_total$style[i])) {
    movie_total$melo[i] <- "1"
  } else {
    movie_total$melo[i] <- "0"
  }
  if(grepl("코미디", movie_total$style[i])) {
    movie_total$comedy[i] <- "1"
  } else {
    movie_total$comedy[i] <- "0"
  }
  if(grepl("성인물", movie_total$style[i])) {
    movie_total$sexual[i] <- "1"
  } else {
    movie_total$sexual[i] <- "0"
  }
  if(grepl("범죄", movie_total$style[i])) {
    movie_total$crime[i] <- "1"
  } else {
    movie_total$crime[i] <- "0"
  }
  if(grepl("공포", movie_total$style[i])) {
    movie_total$horror[i] <- "1"
  } else {
    movie_total$horror[i] <- "0"
  }
  if(grepl("드라마", movie_total$style[i])) {
    movie_total$drama[i] <- "1"
  } else {
    movie_total$drama[i] <- "0"
  }
  if(grepl("어드벤처", movie_total$style[i])) {
    movie_total$adventure[i] <- "1"
  } else {
    movie_total$adventure[i] <- "0"
  }
  if(grepl("미스터리", movie_total$style[i])) {
    movie_total$mystery[i] <- "1"
  } else {
    movie_total$mystery[i] <- "0"
  }
  if(grepl("스릴러", movie_total$style[i])) {
    movie_total$thrill[i] <- "1"
  } else {
    movie_total$thrill[i] <- "0"
  }
  if(grepl("SF", movie_total$style[i])) {
    movie_total$sf[i] <- "1"
  } else {
    movie_total$sf[i] <- "0"
  }
  if(grepl("애니메이션", movie_total$style[i])) {
    movie_total$animation[i] <- "1"
  } else {
    movie_total$animation[i] <- "0"
  }
  if(grepl("판타지", movie_total$style[i])) {
    movie_total$fantasy[i] <- "1"
  } else {
    movie_total$fantasy[i] <- "0"
  }
  if(grepl("가족", movie_total$style[i])) {
    movie_total$family[i] <- "1"
  } else {
    movie_total$family[i] <- "0"
  }
  if(grepl("전쟁", movie_total$style[i])) {
    movie_total$war[i] <- "1"
  } else {
    movie_total$war[i] <- "0"
  }
  if(grepl("뮤지컬", movie_total$style[i])) {
    movie_total$musical[i] <- "1"
  } else {
    movie_total$musical[i] <- "0"
  }
  if(grepl("다큐", movie_total$style[i])) {
    movie_total$daqu[i] <- "1"
  } else {
    movie_total$daqu[i] <- "0"
  }
  if(grepl("공연", movie_total$style[i])) {
    movie_total$play[i] <- "1"
  } else {
    movie_total$play[i] <- "0"
  }
  if(grepl("기타", movie_total$style[i])) {
    movie_total$etc[i] <- "1"
  } else {
    movie_total$etc[i] <- "0"
  }
  if(grepl("서부극", movie_total$style[i])) {
    movie_total$western[i] <- "1"
  } else {
    movie_total$western[i] <- "0"
  }
  
}

In [35]:
# 잘 들어갔는지 확인
head(movie_total, 10)

idx,code,m_name,director,productor,incomer,distributor,open,type1,type2,...,sf,animation,fantasy,family,war,musical,daqu,play,etc,western
1,78726,도둑들,최동훈,(주)케이퍼필름,NA,쇼박스㈜미디어플렉스,2012-07-25,개봉영화,장편,...,0,0,0,0,0,0,0,0,0,0
2,83893,"광해, 왕이된남자",추창민,"리얼라이즈픽쳐스(주),씨제이이앤엠주식회사",NA,씨제이이앤엠주식회사,2012-09-13,개봉영화,장편,...,0,0,0,0,0,0,0,0,0,0
3,72363,어벤져스,조스웨던,NA,한국소니픽쳐스릴리징브에나비스타영화㈜,한국소니픽쳐스릴리징브에나비스타영화㈜,2012-04-26,개봉영화,장편,...,1,0,0,0,0,0,0,0,0,0
4,88253,늑대소년,조성희,(주)영화사비단길,NA,씨제이이앤엠주식회사,2012-10-31,개봉영화,장편,...,0,0,1,0,0,0,0,0,0,0
5,72054,다크나이트라이즈,크리스토퍼놀란,NA,워너브러더스코리아(주),워너브러더스코리아(주),2012-07-19,개봉영화,장편,...,0,0,0,0,0,0,0,0,0,0
6,86888,바람과함께사라지다,김주호,"(주)두타연,(유)에이디사공육",NA,(주)넥스트엔터테인먼트월드(NEW),2012-08-08,개봉영화,장편,...,0,0,0,0,0,0,0,0,0,0
7,66823,어메이징스파이더맨,마크웹,NA,한국소니픽쳐스릴리징브에나비스타영화㈜,한국소니픽쳐스릴리징브에나비스타영화㈜,2012-06-28,개봉영화,장편,...,0,0,0,0,0,0,0,0,0,0
8,82540,범죄와의전쟁: 나쁜놈들전성시대,윤종빈,"(주)팔레트픽처스,쇼박스㈜미디어플렉스",NA,쇼박스㈜미디어플렉스,2012-02-02,개봉영화,장편,...,0,0,0,0,0,0,0,0,0,0
9,89606,내아내의모든것,민규동,"(주)수필름,영화사집",NA,(주)넥스트엔터테인먼트월드(NEW),2012-05-17,개봉영화,장편,...,0,0,0,0,0,0,0,0,0,0
10,88225,연가시,박정우,(주)오죤필름,NA,씨제이이앤엠주식회사,2012-07-05,개봉영화,장편,...,1,0,0,0,0,0,0,0,0,0


In [36]:
# DimStyle 만들기
dim_style <- "CREATE TABLE DimStyle (
                code varchar(255),
                m_name varchar(255),
                style varchar(255),
                action VARCHAR(30),
                history VARCHAR(30),
                melo VARCHAR(30),
                comedy VARCHAR(30),
                sexual VARCHAR(30),
                crime VARCHAR(30),
                horror VARCHAR(30),
                drama VARCHAR(30),
                adventure VARCHAR(30),
                mystery VARCHAR(30),
                thrill VARCHAR(30),
                sf VARCHAR(30),
                animation VARCHAR(30),
                fantasy VARCHAR(30),
                family VARCHAR(30),
                war VARCHAR(30),
                musical VARCHAR(30),
                daqu VARCHAR(30),
                play VARCHAR(30),
                etc VARCHAR(30),
                western VARCHAR(30)
            )"
dbGetQuery(con, dim_style)

<0 x 0 matrix>

In [37]:
# 필요한 컬럼만 추출
movie_total_style <- movie_total[,c("code", "m_name", "style", "action", "history",
                                    "melo", "comedy", "sexual", "crime", "horror", "drama",
                                    "adventure", "mystery", "thrill", "sf", "animation",
                                    "fantasy", "family", "war", "musical", "daqu", "play",
                                    "etc", "western")]
str(movie_total_style)

Classes 'tbl_df', 'tbl' and 'data.frame':	865 obs. of  24 variables:
 $ code     : num  78726 83893 72363 88253 72054 ...
 $ m_name   : chr  "도둑들" "광해, 왕이된남자" "어벤져스" "늑대소년" ...
 $ style    : chr  "액션,범죄" "사극,드라마" "액션,SF" "멜로/로맨스,드라마,판타지" ...
 $ action   : chr  "1" "0" "1" "0" ...
 $ history  : chr  "0" "1" "0" "0" ...
 $ melo     : chr  "0" "0" "0" "1" ...
 $ comedy   : chr  "0" "0" "0" "0" ...
 $ sexual   : chr  "0" "0" "0" "0" ...
 $ crime    : chr  "1" "0" "0" "0" ...
 $ horror   : chr  "0" "0" "0" "0" ...
 $ drama    : chr  "0" "1" "0" "1" ...
 $ adventure: chr  "0" "0" "0" "0" ...
 $ mystery  : chr  "0" "0" "0" "0" ...
 $ thrill   : chr  "0" "0" "0" "0" ...
 $ sf       : chr  "0" "0" "1" "0" ...
 $ animation: chr  "0" "0" "0" "0" ...
 $ fantasy  : chr  "0" "0" "0" "1" ...
 $ family   : chr  "0" "0" "0" "0" ...
 $ war      : chr  "0" "0" "0" "0" ...
 $ musical  : chr  "0" "0" "0" "0" ...
 $ daqu     : chr  "0" "0" "0" "0" ...
 $ play     : chr  "0" "0" "0" "0" ...
 $ etc      : chr

In [38]:
# DimStyle 테이블에 movie_total 데이터 insert
a <- dbWriteTable(con, name = "dimstyle", value = movie_total_style, append = T, row.names=F)
a

[1] TRUE

In [39]:
# DimDate 만들기
dim_date <- "CREATE TABLE DimDate(
                datekey VARCHAR(255),
                year VARCHAR(10),
                month VARCHAR(10),
                day VARCHAR(10),
                datename VARCHAR(10)
            )"
dbGetQuery(con, dim_date)

<0 x 0 matrix>

In [40]:
insert_facttable_to_dimdate <- "INSERT INTO DimDate(datekey)
                                    SELECT DISTINCT datekey
                                    FROM FactTable
                                    ORDER BY datekey"
dbGetQuery(con, insert_facttable_to_dimdate)

<0 x 0 matrix>

In [41]:
# Datekey 일 구분
update_day <- "UPDATE DimDate
                SET day = day(datekey)"
dbGetQuery(con, update_day)

<0 x 0 matrix>

In [42]:
# Datekey 월 구분
update_month <- "UPDATE DimDate
                SET month = month(datekey)"
dbGetQuery(con, update_month)

<0 x 0 matrix>

In [43]:
# Datekey 월 구분
update_year <- "UPDATE DimDate
                SET year = year(datekey)"
dbGetQuery(con, update_year)

<0 x 0 matrix>

In [44]:
# Datekey 요일 구분
update_datename <- "UPDATE DimDate
                SET datename = dayname(datekey)"
dbGetQuery(con, update_datename)

<0 x 0 matrix>

In [45]:
# DimNews 테이블 만들기
dim_news <- "CREATE TABLE DimNews(
                code VARCHAR(255),
                m_name VARCHAR(255),
                news_count FLOAT
            )"
dbGetQuery(con, dim_news)

<0 x 0 matrix>

In [46]:
# DimNews 테이블에 movie_news 데이터 insert
insert_movie_news_to_dimnews <- "INSERT INTO DimNews
                                    SELECT DISTINCT code, m_name, news_count
                                    FROM movie_news"
dbGetQuery(con, insert_movie_news_to_dimnews)

<0 x 0 matrix>

In [47]:
# DimWeather 테이블 만들기
dim_weather <- "CREATE TABLE DimWeather(
                datekey VARCHAR(255),
                rainfall FLOAT,
                rain VARCHAR(30)
                )"
dbGetQuery(con, dim_weather)

<0 x 0 matrix>

In [52]:
# DimWeather 테이블에 movie_weather 데이터 insert
insert_movie_weather_dim_weather <- "INSERT INTO DimWeather (datekey, rainfall)
                                        SELECT datekey, rainfall
                                        FROM movie_weather"
dbGetQuery(con, insert_movie_weather_dim_weather)

<0 x 0 matrix>

In [58]:
# DimWeather 테이블의 잘못 들어간 코드 찾기
mis_data <- "SELECT DISTINCT a.datekey, b.datekey
            FROM DimDate as a right outer join DimWeather as b
                on a.datekey = b.datekey
                where a.datekey is null"
mis_data_check <- dbGetQuery(con, mis_data)
head(mis_data_check)
print(NROW(mis_data_check))

datekey,datekey
NA,2013-01-08
NA,2013-01-09
NA,2013-08-06
NA,2013-08-07
NA,2013-08-08
NA,2013-08-09


[1] 37


In [60]:
# 잘못 들어간 코드 삭제
delete_mis_data <- "DELETE FROM DimWeather
                    WHERE datekey in
                            (SELECT datekey
                            FROM
                                (SELECT DISTINCT b.datekey
                                FROM DimDate as a right outer join DimWeather as b
                                ON a.datekey = b.datekey
                                WHERE a.datekey is null)tmp)"
dbGetQuery(con, delete_mis_data)

<0 x 0 matrix>

In [61]:
# 강수량을 기준으로 비가 왔는지 안왔는지 확인 후 업데이트
update_rainfall_2 <- "UPDATE DimWeather
                        SET rain = '2'
                        WHERE rainfall < 1"
dbGetQuery(con, update_rainfall_2)

<0 x 0 matrix>

In [62]:
# 강수량을 기준으로 비가 왔는지 안왔는지 확인 후 업데이트
update_rainfall_1 <- "UPDATE DimWeather
                        SET rain = '1'
                        WHERE rainfall >= 1"
dbGetQuery(con, update_rainfall_1)

<0 x 0 matrix>

In [30]:
# DB와 연결 종료
# dbDisconnect(con)